# RADIA Example 6: Simple Quadrupole Magnet

In this example, we model a simple iron-dominated quadrupole magnet.
The pole-tips have hyperbolic faces with a chamfer at the ends.

Field computations in the case of iron-dominated geometries present specific
difficulties that usually make them less accurate than those in the case of
structures dominated by coils or permanent magnets. Nevertheless, Radia
includes special methods that enable one to obtain reasonable precision
with a reasonable amount of computational effort—cpu time and memory usage.

Because this example bears some similarities to that of Example&#160;5,
all the remarks made there also apply here. We recommend that you at least
review the introduction of that example before playing with this one.

As a brief reminder, the following recommendations will help you achieve
an acceptable level of precision within a reasonable time:

* Segment the corners of iron circuits as parallel or as perpendicular
as possible to lines of magnetic flux. For right-angled corners, one can
do this using the circular or ellipsoidal mode of segmentation (see below). 
Following this recommendation will have a significant impact on your
simulations of iron-dominated electromagnets.

    In the example shown here, we make use of circular segmentation twice
(with the other two corners addressed by symmetry). See the function
`create_quadrupole(..)` in the section below
entitled **_Define a general function to build a quadrupole magnet_**,
and look for the lines containing

```
rad.ObjDivMag(g3, [nr3, np3, nx], 'cyl', cy)
```

    and
    
```
rad.ObjDivMag(g5, [nr5, np5, nx], 'cyl', cy)
```

* Use a finer segmentation for the iron regions (particularly the pole
pieces) closest to the region of interest. 

* Start with a coarse segmentation and gradually make it finer until
the computed field values are stable.
Be aware that both memory usage and cpu time tend to increase as the
square of the number of elements (segments) in the iron.

* To the greatest extent possible, take advantage of any symmetries
present in your system. Doing so saves both memory usage and CPU time.

For an explanation of all Radia functions, see the
[Radia Reference Guide](
  https://www.esrf.eu/Accelerators/Groups/InsertionDevices/Software/Radia/Documentation/ReferenceGuide/Index
  "RADIA Reference Guide at ESRF").

## _Import Radia and other packages_

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.constants as sc
import numpy # as np
import time as tm
from math import *

import radia as rad
from uti_plot import *
import ipywidgets
from jupyter_rs_radia import radia_viewer

The following figure illustrates the simple dipole steering magnet
we simulate in this example.

In [ ]:
# import an illustration of this magnet
from IPython.display import Image
# Image(filename=('./IronQuadrupole.png'))

## _Define a general function to build a quadrupole focusing magnet_

Here we define a function that creates the quadrupole focusing magnet
of this example. The various arguments (detailed in the function’s
docstring) specify the geometry, material properties, current, and 
segmentation of this model magnet.

In [ ]:
def create_quadrupole():
    '''
    create a simple quadrupole focusing electromagnet
    arguments:
        gap     = distance between ... / mm
        thick   = thickness of iron pole tip (along particle trajectory) / mm
        width   = width of ... / mm
        height  = height of ... / mm
        chamfer = size of chamfer on ends of iron pole tip / mm
        np
        Nn
        Rmin
        ...
    return Radia representation of a simple quadrupole focusing magnet
    '''

    rap = 0.5
    ct = [0, 0, 0]
    z0 = gap / 2
    y0 = width / 2
    amax = hyp * asinh(y0 / z0)
    dz = z0 * (cosh(amax) - 1)
    aStep = amax / np
    na = int(amax * (1 + 2 / np) / aStep) + 1
    qq = [[z0 * sinh(ia * aStep / hyp), z0 * cosh(ia * aStep)] for ia in range(na)]
    hh = qq[np][1] + height * rap - dz
    qq[np + 1] = [qq[np][0], hh]
    qq[np + 2] = [0, hh]
    g1 = rad.ObjThckPgn(thick / 4, thick / 2, qq)   
    rad.ObjDivMag(g1, n1)

    # vertical segment on top of pole faces
    g2 = rad.ObjRecMag([thick / 4, width / 4, gap / 2 + height * (1 / 2 + rap / 2)],
                       [thick / 2, width / 2, height * (1 - rap)])
    rad.ObjDivMag(g2, n2)

    # corner
    gg = rad.ObjCnt([g1, g2])
    gp = rad.ObjCutMag(gg, [thick / 2 - chamfer - gap / 2, 0, 0], [1, 0, -1])[0]
    g3 = rad.ObjRecMag([thick / 4, width / 4, gap / 2 + height + depth / 2],
                       [thick / 2, width / 2, depth])
    cy = [[[0, width / 2, gap / 2 + height], [1, 0, 0]], [0, 0, gap / 2 + height], 2 * depth / width]
    rad.ObjDivMag(g3, [nr3, np3, nx], 'cyl', cy)

    # horizontal segment between the corners
    tan_n = tan(2 * pi / 2 / Nn)
    length = tan_n * (height + gap / 2) - width / 2
    g4 = rad.ObjRecMag([thick / 4, width / 2 + length / 2, gap / 2 + height + depth / 2],
                       [thick / 2,length, depth])
    rad.ObjDivMag(g4, n4)

    # other corner
    posy = width / 2 + length
    posz = posy / tan_n
    g5 = rad.ObjThckPgn(thick / 4, thick / 2, [[posy, posz],[posy, posz + depth], [posy + depth * tan_n, posz + depth]])
    cy = [[[0, posy, posz], [1, 0, 0]], [0, posy, posz + depth], 1]
    rad.ObjDivMag(g5, [nr5, np5, nx], 'cyl', cy)

    # generation of the coil
    Rmax = Rmin - width/2 + gap/2 + offset - 2
    coil1 = rad.ObjRaceTrk([0,0,gap/2+height/2+offset/2], [Rmin,Rmax], [thick,width-2*Rmin], height-offset, 3, CurDens)
    rad.ObjDrwAtr(coil1, coilcolor)
    hh = (height - offset)/2
    coil2 = rad.ObjRaceTrk([0,0,gap/2+height-hh/2], [Rmax,Rmax+hh*0.8], [thick,width-2*Rmin], hh, 3, CurDens)
    rad.ObjDrwAtr(coil2, coilcolor)

    # make container, set colors, and define symmetries
    g = rad.ObjCnt([gp, g3, g4, g5])
    rad.ObjDrwAtr(g, ironcolor)
    gd = rad.ObjCnt([g])
    
    rad.TrfZerPerp(gd, ct, [1, 0, 0])
    rad.TrfZerPerp(gd, ct, [0, 1, 0])
    
    t = rad.ObjCnt([gd, coil1, coil2])
    rad.TrfZerPara(t, ct, [0, cos(pi / Nn), sin(pi / Nn)])

    rad.TrfMlt(t, rad.TrfRot(ct, [1, 0, 0], 4 * pi / Nn), int(round(Nn / 2)))
    rad.MatApl(g, ironmat)
    rad.TrfOrnt(t, rad.TrfRot([0, 0, 0],[1, 0, 0],pi / Nn))
    
    return t

In [ ]:
# harmonic analysis

def harm(obj, y, z, ro, np):
    '''
    perform ...
    arguments:
        obj = 
        y   = 
        z   = 
        ro  = 
        np  = 
    return list [, , ]
    '''
    arHarm = [complex(0,0)] * np
    d_tet = 2*pi/np
    tet = 0
    for i in range(np):
        cosTet = cos(tet); sinTet = sin(tet)
        re = rad.FldInt(obj, 'inf', 'ibz', [-1,y+ro*cosTet,z+ro*sinTet], [1,y+ro*cosTet,z+ro*sinTet])
        im = rad.FldInt(obj, 'inf', 'iby', [-1,y+ro*cosTet,z+ro*sinTet], [1,y+ro*cosTet,z+ro*sinTet])
        arHarm[i] = complex(re, im)
        tet += d_tet
    return [np, ro, arHarm]

def multipole(w, q):
    np = w[0]; ro = w[1]; arHarm = w[2]
    s = 0
    for p in range(np):
        arg = -2*pi/np*q*p
        s += arHarm[p]*complex(cos(arg), sin(arg))
    return s/np/(ro**q)    

## _Create the quadrupole and solve for the fields_

First set the various parameters that specify the properties—geometry,
materials, and current—of our quadrupole. Then also decide how finely
to segment the iron.

In [ ]:
# general parameters for quadrupole geometry
np = 4
hyp = 1      # must be >0.2 !
gap = 40     # magnetic gap
width = 30   # pole width
height = 50  # height of iron yoke
thick = 60   # length of the quad
chamfer = 8  # size of chamfer

Nn = 4
ironcolor = [0.0, 1.0, 1.0]
coilcolor = [1.0, 0.0, 0.0]
depth = 1.2*width/2

#Coils
CurDens = -3 #current density [A/mm^2]
Rmin = 2
offset = 10

#Segmentation Params
nx = 2
ny = 2
n1 = [nx,ny,3]
n2 = [nx,ny,3]
np3 = 2
nr3 = ny
n4 = [nx,3,ny]
np5 = ceil(np3/2); print()
nr5 = ny

Now create and display this quadrupole magnet:

In [ ]:
rad.UtiDelAll()
ironmat = rad.MatSatIsoFrm([2000,2],[0.1,2],[0.1,2])
t0 = tm.time()
quad = create_quadrupole()
t1 = tm.time()
size = rad.ObjDegFre(quad)

print('Built in time', round((t1 - t0) * 1e3, 3),'ms')
print('Interaction matrix:', size, 'x', size, '.equiv.', (4 * size * size / 1e6), 'MBytes')

# set up the radia viewer and display the magnet
rv = radia_viewer.RadiaViewer()
rv.add_geometry('Quadrupole Magnet', quad)
rv.display()

In [ ]:
# solve for the magnetization
prec     = 10.e-6 # precision for this computation
max_iter = 10000  # maximum allowed iterations
t0  = tm.time()
res = rad.Solve(quad, prec, max_iter)
t1  = tm.time()

print("Solved for magnetization in time {0:6f} s".format(t1 - t0))
print("Relaxation results")
print("  number of iterations: {0:5d}".format(int(res[3])))
if(res[3] == max_iter):
    print("    >> unstable or incomplete relaxation")
print("  average stability of magnetization at last iteration: {0:.4e} T".format(res[0]))
print("  maximum absolute magnetization at last iteration: {0:.5f} T".format(res[1]))
print("  maximum H vector at last iteration: {0:.5f} T".format(res[2]))
# print("Pole-tip magnetic field: {0:.8f} T".format(rad.Fld(quad, 'bz', [x,y,z])))b

In [ ]:
The quadrupole gradient ... $B_z$ field at the origin, $(0, 0, 0)$, is the vertical
field in the middle of the gap.

In [ ]:
Bz  = rad.Fld(quad, 'Bz', [0, 1, 0]) * 1e3
Bz1 = rad.Fld(quad, 'Bz', [0,10, 0]) * 1e3 / 10
Iz  = rad.FldInt(quad, 'inf', 'ibz', [-1, 1, 0], [1, 1, 0])
Iz1 = rad.FldInt(quad, 'inf', 'ibz', [-1,10, 0], [1,10, 0]) / 10

print(' quadrupole gradient:', round(Bz, 4), 'T/m')
print('  int.quad. at  1 mm:', round(Iz, 5), 'T')
print('δ int.quad. at 10 mm:', round((Iz1 / Iz - 1), 6) * 100, '%') # rel. var. in field integral

## _Plots of the magnetic field and field integrals_

Here we show plots of magnetic field in the gap and corresponding
field integrals. The field values are obtained by calling `Fld` on
a list of points. One may also use `FldLst`.

The first graph here displays ...

In [ ]:
# plots of magnetic field
z    = 0
x1   = 0
x2   = 30
ymax = 40
np   = 20

Bz1 = rad.FldLst(quad, 'bz', [x1, 0, z], [x1, ymax, z], np, 'arg', 0)
Bz2 = rad.FldLst(quad, 'bz', [x2, 0, z], [x2, ymax, z], np, 'arg', 0)

uti_plot1d_m([Bz1, Bz2],
             labels = ['Y', 'Vertical Magnetic Field', 'Vertical Magnetic Field vs. Vertical Position'], units = ['mm', 'T'],
             styles = ['-b.', '--r.'], legend = ['X = {} mm'.format(x1), 'X = {} mm'.format(x2)])

# plots of magnetic field integrals
z    = 0
ymin = 0.001
ymax = 10
npy  = 20
yStep = (ymax - ymin) / (npy - 1)
IBz0 = rad.FldInt(quad, 'inf', 'ibz', [-1, 1, z], [1, 1, z])

IBzVsY = [(rad.FldInt(quad, 'inf', 'ibz', [-1, ymin + iy * yStep, z], [1,ymin+iy*yStep,z]) /
           ((ymin+iy*yStep)*IBz0) - 1) * 100 for iy in range(npy)]
uti_plot1d(IBzVsY, [ymin, ymax, npy],
           ['Y', 'dIz', 'Rel. Variation of Vertical Field Integral along X at Z = ' + repr(z) + ' mm'], ['mm', '%'])

# harmonic analysis of the field integrals
nharm = 10; radius = 2; y = 0; z = 0;
w = harm(quad, y, z, radius, nharm);

mm = [multipole(w, i) for i in range(nharm)];
round_mm = [complex(round(mm[i].real, 9), round(mm[i].imag, 9)) for i in range(nharm)];
print(round_mm)

uti_plot_show()